In [1]:
import tweepy
import pandas as pd
client = tweepy.Client('<BearerToken>')
import pandas as pd
import numpy as np
import os


In [ ]:
def get_tweets_by_userID_list(users_id):
    
    dictionary = {}
    for i in users_id:
        tweetlist =[]
        usertweet = client.get_users_tweets(id=i,max_results=100)
        for tweet in usertweet.data:
            tweetlist.append(tweet.text)
        dictionary[i] = tweetlist
    return(dictionary)

print(get_tweets_by_userID_list(['487297085']))

In [ ]:
#Tweets by one Obama account
def check_if_active_and_valid(user_id: str):
    """This functions returns False if the user has not been active since October 1 of 2022,
    True otherwise.  It should also return false for bad (not numeric) IDs.
    """
    try:
        check = client.get_users_tweets(id=user_id,max_results=10, start_time='2022-10-01T00:00:00Z')
    except:
        return False
    return (not check.data is None)

print(get_tweets_by_userID_list(['216776631', '29442313']))


In [ ]:
#This function does not work yet

def pol_data_cleaner_by_last_active(df):
    """This function will take a data frame representing twitter
    users, and will take out anyone that has not been active
    since Oct 1 2022.
    """
    df = df.dropna(how='any',axis=0) 
    ID_list = list(df['Account_ID'])
    ID_list_str = [str(x) for x in ID_list]
    df['Active'] = [check_if_active_and_valid(x) for x in ID_list_str]
    clean_df = df[df['Active'] == True]
    clean_df = clean_df.drop(['Active'], axis=1)
    return clean_df

data = pd.read_csv('cleaner_politicians.csv')
clean = pol_data_cleaner_by_last_active(data)
print(clean)
clean.to_csv('cleanest_politicians.csv')




In [ ]:
def confirm_its_clean(df):
    from collections import Counter
    """Checks that there are no duplcates IDs
    or name in our csv."""
    name_list = list(df['Name'])
    return Counter(name_list)

print(confirm_its_clean(pd.read_csv('cleanest_politicians.csv')))
    

In [ ]:
import re
from cleantext import clean
def twitter(users_id):
    """Script for gathering raw tweets for a list of user id"""
    dictionary = {}
    for i in users_id:
        tweetlist =[]
        usertweet = client.get_users_tweets(id=i,max_results=10)
        for tweet in usertweet.data:
            #get rid of emoji
            cleantext = clean(str(tweet.text))
            # remove tab
            text = cleantext.replace('\n',' ')
            # get rid of url/links
            rawtwt = re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)
            #append raw tweet
            tweetlist.append(rawtwt)
        dictionary[i] = tweetlist
    return(dictionary)

print(twitter(['216776631']))

In [ ]:
#Paginator testing

counter = 0
for tweet in tweepy.Paginator(client.get_users_tweets, "216776631",
                                max_results=10).flatten(limit=10):
    print(tweet.id)
    counter += 1

print(counter)

#It works, we can get a lot more tweets per user this way


In [ ]:
#Get tweets with pagination, put them into a csv

import re
from cleantext import clean
def get_paged_tweets(user_id):
    """Script for gathering raw tweets for a list of user id and saving them to csv"""
    df = pd.DataFrame(columns=['TweetID', 'UserID', 'Text'])
    tweetlist =[]
        # usertweet_page = tweepy.Paginator(client.get_users_tweets, i,
                                # max_results=10).flatten(limit=10)
        # print(usertweet_page.id)
    for tweet in tweepy.Paginator(client.get_users_tweets, user_id, exclude='retweets',
                                  max_results=100).flatten(limit=1000):
        #     #get rid of emoji
        # tweet_text = tweet.text.lower()
        # clean_text = clean(str(tweet_text))
        #     # remove tab
        text = tweet.text.replace('\n',' ')
        # print(text)
        #     # get rid of url/links
        # rawtwt = re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)
        #     #append raw tweet
        df.loc[len(df.index)] = [tweet.id, user_id, text]
    return df

print(get_paged_tweets('487297085'))

In [ ]:
#construct csvs for each politician
from pathlib import Path

def fetch_tweets(ID_list, name_list):
    
    # df = pd.read_csv('cleanest_politicians.csv')
    # ID_list = list(df['Account_ID'])
    # name_list = list(df['Name'])

        
    for i in range(0, len(ID_list)):
        file = Path('/Users/dustinhayes/Desktop/STAT766Final/fixed_pol_csvs/{}-{}.csv'.format((ID_list[i]), name_list[i]))
        print('getting {}'.format(name_list[i]))
        if file.exists():
            pass
        else:
            df = get_paged_tweets(ID_list[i])
            df.to_csv('/Users/dustinhayes/Desktop/STAT766Final/fixed_pol_csvs/{}-{}.csv'.format((ID_list[i]), name_list[i]))

fetch_tweets(['80013913', '256074273', '487297085', '854715071116849157', '233737858'], ['Ricardo Rossello', 'Roland Gtierrez', 'Ron DeSantis', 'Ron Estes', 'Ron John'])

In [ ]:
#Filter csvs such that only pol with X or more tweets included

def make_pol_csvs_min_only(csv_path: str, min_tweets: int):
    df = pd.read_csv(csv_path)
    if len(list(df['TweetID'])) >= min_tweets:
        df.to_csv('{}\\{}'.format(min_tweets))
        print(csv_path)

os.chdir('/Users/dustinhayes/Desktop/STAT766Final/politician_csvs')
pol_csvs = os.listdir()
for i in range(0, len(pol_csvs[0:5])):
    make_pol_csvs_min_only(pol_csvs[i], 1000)
